In [1]:
import numpy as np
import pandas as pd
import umap

In [3]:
ivs_df = pd.read_pickle("../data/ivs_df.pkl")
country_codes = pd.read_pickle("../data/country_codes.pkl")

############################################
########Data Preperation#################
############################################

# Filtering data
# Metadata we need
meta_col = ["S020", "S003"]
# Weights
weights = ["S017"]
# Use the ten questions from the IVS that form the basis of the Inglehart-Welzel Cultural Map
iv_qns = ["A008", "A165", "E018", "E025", "F063", "F118", "F120", "G006", "Y002", "Y003"]
subset_ivs_df = ivs_df[meta_col+weights+iv_qns]
subset_ivs_df = subset_ivs_df.rename(columns={'S020': 'year', 'S003': 'country_code', 'S017': 'weight'})
# remove data from before 2005
# We need to filter down to the three most recent survey waves (from 2005 onwards). The most recent survey waves provide up-to-date information on cultural values, ensuring that the analysis reflects current societal norms and attitudes. We also filter out the ten questions from the IVS that form the basis of the Inglehart-Welzel Cultural Map.
subset_ivs_df = subset_ivs_df[subset_ivs_df["year"] >= 2005]

In [8]:
# Take the first 20 rows of the dataset
subset_ivs_df = subset_ivs_df.head(20).copy()

In [9]:
subset_ivs_df[iv_qns] = subset_ivs_df[iv_qns].multiply(subset_ivs_df["weight"], axis=0)

In [18]:
subset_ivs_df.dropna(subset=iv_qns, thresh=10)

,year,country_code,weight,A008,A165,E018,E025,F063,F118,F120,G006,Y002,Y003
1,2018.0,20.0,1.0,1.0,2.0,1.0,1.0,1.0,9.0,9.0,1.0,3.0,-1.0
2,2018.0,20.0,1.0,2.0,2.0,1.0,1.0,8.0,10.0,5.0,3.0,2.0,2.0
7,2018.0,20.0,1.0,2.0,2.0,2.0,1.0,4.0,10.0,6.0,1.0,3.0,-1.0
10,2018.0,20.0,1.0,2.0,2.0,2.0,1.0,10.0,7.0,4.0,1.0,3.0,2.0
11,2018.0,20.0,1.0,1.0,1.0,1.0,1.0,8.0,1.0,7.0,2.0,3.0,0.0
13,2018.0,20.0,1.0,2.0,2.0,2.0,1.0,6.0,10.0,10.0,1.0,2.0,1.0
16,2018.0,20.0,1.0,2.0,2.0,2.0,1.0,1.0,10.0,5.0,2.0,2.0,1.0
17,2018.0,20.0,1.0,2.0,2.0,3.0,1.0,5.0,10.0,10.0,2.0,2.0,0.0
18,2018.0,20.0,1.0,1.0,2.0,2.0,1.0,10.0,6.0,1.0,1.0,2.0,0.0


In [13]:
data = subset_ivs_df[iv_qns].to_numpy()
data

array([[ 1.,  2.,  1.,  2.,  7.,  4.,  2., nan,  1.,  0.],
       [ 1.,  2.,  1.,  1.,  1.,  9.,  9.,  1.,  3., -1.],
       [ 2.,  2.,  1.,  1.,  8., 10.,  5.,  3.,  2.,  2.],
       [ 2.,  2.,  2.,  2.,  1.,  6.,  4., nan,  2.,  0.],
       [ 2.,  2.,  2.,  1.,  4.,  8.,  5., nan,  1.,  1.],
       [ 1.,  2.,  1.,  1.,  3., 10.,  1., nan,  2.,  0.],
       [ 2.,  2.,  1.,  1.,  8., 10., 10., nan,  3.,  0.],
       [ 2.,  2.,  2.,  1.,  4., 10.,  6.,  1.,  3., -1.],
       [ 1.,  2.,  2.,  2.,  5.,  8.,  5., nan,  3.,  1.],
       [ 1.,  2.,  2.,  2., 10., 10.,  8., nan,  2.,  1.],
       [ 2.,  2.,  2.,  1., 10.,  7.,  4.,  1.,  3.,  2.],
       [ 1.,  1.,  1.,  1.,  8.,  1.,  7.,  2.,  3.,  0.],
       [ 1.,  2.,  2.,  1.,  1., 10., 10., nan,  3.,  2.],
       [ 2.,  2.,  2.,  1.,  6., 10., 10.,  1.,  2.,  1.],
       [ 3.,  2.,  1.,  2.,  8., 10., 10., nan,  1.,  1.],
       [ 2.,  2.,  3.,  2., 10., 10.,  1., nan,  1., -1.],
       [ 2.,  2.,  2.,  1.,  1., 10.,  5.,  2.,  2.,  1.

In [14]:
# Randomly mask 10% of the data
mask = np.random.choice([True, False], data.shape, p=[0.1, 0.9])
data[mask] = np.nan
data

array([[ 1.,  2.,  1.,  2.,  7.,  4., nan, nan,  1.,  0.],
       [ 1.,  2.,  1.,  1., nan,  9.,  9.,  1.,  3., -1.],
       [ 2.,  2.,  1.,  1.,  8., 10., nan,  3.,  2.,  2.],
       [ 2.,  2.,  2.,  2.,  1.,  6.,  4., nan,  2.,  0.],
       [ 2.,  2.,  2.,  1.,  4.,  8.,  5., nan,  1.,  1.],
       [ 1.,  2.,  1.,  1.,  3., 10.,  1., nan,  2.,  0.],
       [ 2.,  2.,  1.,  1.,  8., 10., 10., nan,  3.,  0.],
       [ 2.,  2.,  2., nan,  4., 10.,  6.,  1., nan, -1.],
       [nan,  2.,  2.,  2.,  5.,  8.,  5., nan,  3.,  1.],
       [ 1.,  2., nan,  2., 10., 10.,  8., nan,  2.,  1.],
       [ 2.,  2.,  2.,  1., nan,  7.,  4.,  1.,  3.,  2.],
       [ 1.,  1.,  1.,  1.,  8.,  1.,  7.,  2.,  3.,  0.],
       [ 1.,  2.,  2.,  1., nan, 10., 10., nan,  3.,  2.],
       [ 2.,  2.,  2.,  1.,  6., 10., 10.,  1.,  2.,  1.],
       [ 3.,  2.,  1.,  2.,  8., 10., 10., nan,  1.,  1.],
       [ 2.,  2.,  3.,  2., 10., 10.,  1., nan,  1., -1.],
       [ 2.,  2.,  2., nan,  1., 10.,  5.,  2.,  2., nan

In [15]:
# Determine which data points are observed
observed = ~np.isnan(data)
missing = np.sum(~observed)
data[~observed] = 0

In [16]:
data

array([[ 1.,  2.,  1.,  2.,  7.,  4.,  0.,  0.,  1.,  0.],
       [ 1.,  2.,  1.,  1.,  0.,  9.,  9.,  1.,  3., -1.],
       [ 2.,  2.,  1.,  1.,  8., 10.,  0.,  3.,  2.,  2.],
       [ 2.,  2.,  2.,  2.,  1.,  6.,  4.,  0.,  2.,  0.],
       [ 2.,  2.,  2.,  1.,  4.,  8.,  5.,  0.,  1.,  1.],
       [ 1.,  2.,  1.,  1.,  3., 10.,  1.,  0.,  2.,  0.],
       [ 2.,  2.,  1.,  1.,  8., 10., 10.,  0.,  3.,  0.],
       [ 2.,  2.,  2.,  0.,  4., 10.,  6.,  1.,  0., -1.],
       [ 0.,  2.,  2.,  2.,  5.,  8.,  5.,  0.,  3.,  1.],
       [ 1.,  2.,  0.,  2., 10., 10.,  8.,  0.,  2.,  1.],
       [ 2.,  2.,  2.,  1.,  0.,  7.,  4.,  1.,  3.,  2.],
       [ 1.,  1.,  1.,  1.,  8.,  1.,  7.,  2.,  3.,  0.],
       [ 1.,  2.,  2.,  1.,  0., 10., 10.,  0.,  3.,  2.],
       [ 2.,  2.,  2.,  1.,  6., 10., 10.,  1.,  2.,  1.],
       [ 3.,  2.,  1.,  2.,  8., 10., 10.,  0.,  1.,  1.],
       [ 2.,  2.,  3.,  2., 10., 10.,  1.,  0.,  1., -1.],
       [ 2.,  2.,  2.,  0.,  1., 10.,  5.,  2.,  2.,  0.